In [ ]:
import mdp, util

from learningAgents import ValueEstimationAgent
import collections

class ValueIterationAgent(ValueEstimationAgent):
    """
        * Please read learningAgents.py before reading this.*

        A ValueIterationAgent takes a Markov decision process
        (see mdp.py) on initialization and runs value iteration
        for a given number of iterations using the supplied
        discount factor.
    """
    def __init__(self, mdp, discount = 0.9, iterations = 100):
        """
          Your value iteration agent should take an mdp on
          construction, run the indicated number of iterations
          and then act according to the resulting policy.

          Some useful mdp methods you will use:
              mdp.getStates()
              mdp.getPossibleActions(state)
              mdp.getTransitionStatesAndProbs(state, action)
              mdp.getReward(state, action, nextState)
              mdp.isTerminal(state)
        """
        self.mdp = mdp
        self.discount = discount
        self.iterations = iterations
        self.values = util.Counter() # A Counter is a dict with default 0
        self.runValueIteration()

    def runValueIteration(self):
        # Write value iteration code here
        "*** YOUR CODE HERE ***"
        for iteration in range(self.iterations):
            temp = util.Counter()
            for state in self.mdp.getStates():
                #the value for terminal state is 0
                if self.mdp.isTerminal(state):
                    temp[state] = 0
                else:
                    #get actions and rewards
                    maximumValue = -99999
                    #actions for the state
                    actions = self.mdp.getPossibleActions(state)
                    for action in actions:
                        #find state and probability for the transition associated
                        #with actions
                        t = self.mdp.getTransitionStatesAndProbs(state, action)
                        value = 0
                        for stateAndProb in t:
                            value += stateAndProb[1] * (self.mdp.getReward(state, action, stateAndProb[1]) \
                            + self.discount * self.values[stateAndProb[0]])
                        maximumValue = max(value, maximumValue)
                    if maximumValue != -99999:
                        temp[state] = maximumValue
            self.values = temp


    def getValue(self, state):
        """
          Return the value of the state (computed in __init__).
        """
        return self.values[state]

    def getPolicy(self, state):
        return self.computeActionFromValues(state)

    def getAction(self, state):
        "Returns the policy at the state (no exploration)."
        return self.computeActionFromValues(state)

class AsynchronousValueIterationAgent(ValueIterationAgent):
    """
        * Please read learningAgents.py before reading this.*

        An AsynchronousValueIterationAgent takes a Markov decision process
        (see mdp.py) on initialization and runs cyclic value iteration
        for a given number of iterations using the supplied
        discount factor.
    """
    def __init__(self, mdp, discount = 0.9, iterations = 1000):
        """
          Your cyclic value iteration agent should take an mdp on
          construction, run the indicated number of iterations,
          and then act according to the resulting policy. Each iteration
          updates the value of only one state, which cycles through
          the states list. If the chosen state is terminal, nothing
          happens in that iteration.

          Some useful mdp methods you will use:
              mdp.getStates()
              mdp.getPossibleActions(state)
              mdp.getTransitionStatesAndProbs(state, action)
              mdp.getReward(state)
              mdp.isTerminal(state)
        """
        ValueIterationAgent.__init__(self, mdp, discount, iterations)

    def runValueIteration(self):
        "*** YOUR CODE HERE ***"

        states = self.mdp.getStates()

        for iteration in range(self.iterations):
            state  = states[iteration % len(states)]
            if not self.mdp.isTerminal(state):
                actions = self.mdp.getPossibleActions(state)
                maximumValue = max([self.getQValue(state,action) for action in actions])
                self.values[state] = maximumValue


class PrioritizedSweepingValueIterationAgent(AsynchronousValueIterationAgent):
    """
        * Please read learningAgents.py before reading this.*

        A PrioritizedSweepingValueIterationAgent takes a Markov decision process
        (see mdp.py) on initialization and runs prioritized sweeping value iteration
        for a given number of iterations using the supplied parameters.
    """
    def __init__(self, mdp, discount = 0.9, iterations = 100, theta = 1e-5):
        """
          Your prioritized sweeping value iteration agent should take an mdp on
          construction, run the indicated number of iterations,
          and then act according to the resulting policy.
        """
        self.theta = theta
        ValueIterationAgent.__init__(self, mdp, discount, iterations)

    def runValueIteration(self):
        "*** YOUR CODE HERE ***"

        pQueue = util.PriorityQueue()

        predecessors = {}

        #fid all predecessors
        for state in self.mdp.getStates():
            if not self.mdp.isTerminal(state):
                for action in self.mdp.getPossibleActions(state):
                    for stateAndProb in self.mdp.getTransitionStatesAndProbs(state, action):
                        if stateAndProb[0] in predecessors:
                            predecessors[stateAndProb[0]].add(state)
                        else:
                            predecessors[stateAndProb[0]] = {state}


        for state in self.mdp.getStates():
            if not self.mdp.isTerminal(state):
                diff = abs(self.values[state] - max([ \
                self.computeQValueFromValues(state, action) for action in \
                self.mdp.getPossibleActions(state) ]) )
                #pushing negative into queue
                pQueue.update(state, -diff)

        for iteration in range(self.iterations):
            if pQueue.isEmpty():
                break
            state = pQueue.pop()
            if not self.mdp.isTerminal(state):
                self.values[state] = max([self.computeQValueFromValues(state, action)\
                 for action in self.mdp.getPossibleActions(state)])

            for p in predecessors[state]:
                if not self.mdp.isTerminal(p):
                    diff = abs(self.values[p] - max([self.computeQValueFromValues(p, action)\
                     for action in self.mdp.getPossibleActions(p)]))

                    if diff > self.theta:
                            pQueue.update(p, -diff)
